In [2]:
from deepface import DeepFace
from deepface.modules import verification
import numpy as np

from deepface.models.FacialRecognition import FacialRecognition
import cv2

import ezkl
import os
import json

import tf2onnx
import onnx

import torch
from onnx2torch import convert

In [3]:
model_name = "OpenFace"
model: FacialRecognition = DeepFace.build_model(model_name)

onnx_model, _ = tf2onnx.convert.from_keras(model.model)
onnx.save(onnx_model, "openface_new.onnx")
pytorch_model = convert(onnx_model)

pytorch_model.eval()

target_size = model.input_shape

In [4]:
img1 = DeepFace.extract_faces(img_path="emmapassport.png")[0]["face"]
img1 = cv2.resize(img1, target_size)
img1 = np.expand_dims(img1, axis=0)  # to (1, 224, 224, 3)

img1_torch = torch.tensor(img1, dtype=torch.float32)

In [5]:
# Export the model
torch.onnx.export(pytorch_model,               # model being run
                      img1_torch,                   # model input (or a tuple for multiple inputs)
                      "openface_pt.onnx",            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=10,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})


In [6]:
data_array = (img1).reshape([-1]).tolist()
data = dict(input_data = [data_array])
data_path = os.path.join('input.json')
settings_path = os.path.join('settings.json')
model_path = os.path.join('openface_pt.onnx')

In [7]:
# Serialize data into file:
json.dump( data, open(data_path, 'w' ))

py_run_args = ezkl.PyRunArgs()
py_run_args.input_visibility = "private"
py_run_args.output_visibility = "public"
py_run_args.param_visibility = "fixed" # private by default

# What is the settings file?
res = ezkl.gen_settings(model_path, settings_path, py_run_args=py_run_args)

assert res == True

In [8]:
cal_path = os.path.join("calibration.json")

img1 = DeepFace.extract_faces(img_path="kayleepassport.jpg")[0]["face"]
img1 = cv2.resize(img1, target_size)
img1 = np.expand_dims(img1, axis=0)  # to (1, 224, 224, 3)

data_array = (img1).reshape([-1]).tolist()
data = dict(input_data = [data_array])

json.dump(data, open(cal_path, 'w'))

ezkl.calibrate_settings(cal_path, model_path, settings_path, target="resources")

<Future pending cb=[<builtins.PyDoneCallback object at 0x2991fa210>()]>

In [10]:
compiled_model_path = os.path.join('network.compiled')

In [12]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)

In [13]:
assert res == True

In [14]:
# srs path
res = await ezkl.get_srs( settings_path)

In [15]:
# now generate the witness file 
witness_path = os.path.join('witness.json')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')

res = await ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

In [ ]:
# HERE WE SETUP THE CIRCUIT PARAMS
# WE GOT KEYS
# WE GOT CIRCUIT PARAMETERS
# EVERYTHING ANYONE HAS EVER NEEDED FOR ZK



res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
        
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

PanicException: dynamic lookup or shuffle should only have one block

Using 5 columns for non-linearity table.
Using 5 columns for non-linearity table.
Using 5 columns for non-linearity table.
Using 5 columns for non-linearity table.
Using 5 columns for non-linearity table.
Using 5 columns for non-linearity table.
Using 9 columns for non-linearity table.
Using 9 columns for non-linearity table.
Using 9 columns for non-linearity table.
Using 9 columns for non-linearity table.
Using 9 columns for non-linearity table.
Using 9 columns for non-linearity table.
calibration failed extended k is too large to accommodate the quotient polynomial with logrows 20
circuit creation from run args failed: SigBitTruncationError
circuit creation from run args failed: SigBitTruncationError
calibration failed max lookup input (-149327430, 702515025) is too large
calibration failed max lookup input (-79466365611640, 3105515024213904) is too large
calibration failed max lookup input (-124884517, 426728673) is too large
circuit creation from run args failed: SigBitTruncationEr